In [7]:

from requests import post
from pydantic import BaseModel
from trilogy import Dialects
from typing import List, Optional

In [8]:
class ModelSourceInSchema(BaseModel):
    alias: str
    contents: str


class ModelInSchema(BaseModel):
    name: str
    sources: list[ModelSourceInSchema]
    

class Import(BaseModel):
    name: str
    alias: str | None = None


class QueryInSchema(BaseModel):
    imports: list[Import]
    query: str
    dialect: Dialects
    full_model: ModelInSchema
    extra_filters: Optional[list[str]] = None
    parameters: Optional[dict[str, str | int | float]] = None

In [11]:
full_model = {
  "name": "",
  "sources": [
    {
      "alias": "customer",
      "contents": "import std.money;\nimport nation as nation;\n\nkey id int;\nproperty id.name string;\nproperty id.address string;\nproperty id.phone string;\nproperty id.account_balance float::usd;\nproperty id.market_segment string;\nproperty id.comment string;\n\ndatasource customers (\n    c_custkey:id,\n    c_name:name,\n    c_address:address,\n    c_nationkey:nation.id,\n    c_phone:phone,\n    c_acctbal:account_balance,\n    c_mktsegment:market_segment,\n    c_comment:comment\n)\ngrain (id)\naddress customer;"
    },
    {
      "alias": "lineitem",
      "contents": "import std.money;\nimport std.display;\nimport order as order;\nimport supplier as supplier;\nimport part as part;\n\n\n\nkey id int; #unique key identifying a lineitem within an order\nproperty id.quantity float; #quantity of an part within an order\nproperty id.extended_price float::usd;\nproperty id.discount float::percent; # percent discount\nproperty id.tax float::percent; # tax, as percentage\nproperty id.return_flag string;\nproperty id.line_status string;\nproperty id.ship_date date;\nproperty id.commit_date date;\nproperty id.receipt_date date;\nproperty id.ship_instruct string;\nproperty id.ship_mode string;\nproperty id.comment string;\n\n\nproperty id.revenue <- extended_price* (1 - discount); # revenue is our price, with discount applied\n\nproperty id.customer_charged <- revenue * (1 + tax); # the customer is charged a total that includes tax based on the revenue\n\nauto total_revenue <- sum(revenue); # total revenue is an aggregated revenue suitable for use in a report\n\n# upper some fileds to fix stats issue\ndatasource lineitem(\n    l_orderkey: order.id,\n    l_partkey: part.id,\n    l_suppkey: supplier.id,\n    l_linenumber:id,\n    l_quantity:quantity,\n    l_extendedprice: extended_price,\n    l_discount:discount,\n    l_tax:tax,\n    l_returnflag:return_flag,\n    l_linestatus: line_status,\n    l_shipdate: ship_date,\n    l_commitdate: commit_date,\n    l_receiptdate:receipt_date,\n    l_shipinstruct:ship_instruct,\n    l_shipmode: ship_mode,\n    l_comment:comment\n)\ngrain(id)\naddress lineitem;"
    },
    {
      "alias": "nation",
      "contents": "import std.geography;\nimport region as region;\n\nkey id int;\nproperty id._raw_name string; #internal field used to workaround issues with parquet statistics\nproperty id.name string::country; # capitalized; eg UNITED STATES or FRANCE or CANADA\nproperty id.comment string;\n\ndatasource nation (\n    n_nationkey:id,\n    n_name: _raw_name,\n    UPPER(_raw_name):name,\n    n_regionkey:region.id,\n    n_comment:comment\n)\ngrain(id)\naddress nation;\n"
    },
    {
      "alias": "order",
      "contents": "import std.money;\n\nimport customer as customer;\n\nkey id int; #order key\nproperty id.status string;\nproperty id.total_price float::usd;\nproperty id.date date;\nproperty id.priority string;  #enum, one of: 1-URGENT, 2-HIGH, 3-MEDIUM, 4-NOT SPECIFIED\nproperty id.clerk string;\nproperty id.ship_priority string;  #enum, one of: 1-URGENT, 2-HIGH, 3-MEDIUM, 4-NOT SPECIFIED\nproperty id.comment string;\n\ndatasource orders(\n    o_orderkey:id,\n    o_custkey:customer.id,\n    o_orderstatus: status,\n    o_totalprice: total_price,\n    o_orderdate: date,\n    o_orderpriority:priority, \n    o_clerk:clerk,\n    o_comment:comment,\n    o_shippriority:ship_priority \n)\ngrain (id)\naddress orders;"
    },
    {
      "alias": "part",
      "contents": "import std.money;\n\nimport supplier as supplier;\n\nkey id int;\nproperty id._name string; #internal field used to workaround issues with parquet statistics\nproperty id.name string;\nproperty id.manufacturer string;\nproperty id.brand string;\nproperty id.type string;\nproperty id.size int;\nproperty id.container string;\nproperty id.retail_price float::usd;\nproperty id.comment string;\n\ndatasource part (\n    p_partkey:id,\n    p_name:_name,\n    UPPER(_name):name,\n    p_mfgr: manufacturer,\n    p_brand:brand,\n    p_type:type,\n    p_size:size,\n    p_container:container,\n    p_retailprice:retail_price,\n    p_comment:comment\n)\ngrain(id)\naddress part;\n\nproperty <id,supplier.id>.available_quantity float;\nproperty <id,supplier.id>.supply_cost float::usd;\nproperty <id,supplier.id>.supplier_comment string;\n\ndatasource partsupp (\n    ps_partkey:id,\n    ps_suppkey:supplier.id,\n    ps_availqty: available_quantity,\n    ps_supplycost:supply_cost,\n    ps_comment:supplier_comment\n)\ngrain (id, supplier.id)\naddress partsupp;"
    },
    {
      "alias": "region",
      "contents": "key id int;\nproperty id.name string; # capitalized; eg ASIA or EUROPE or MIDDLE EAST etc\nproperty id.comment string;\n\ndatasource region (\n    r_regionkey:id,\n    r_name:name ,\n    r_comment:comment\n)\ngrain(id)\naddress region;"
    },
    {
      "alias": "supplier",
      "contents": "import nation as nation;\n\nkey id int;\nproperty id.name string;\nproperty id.address string;\nproperty id.phone string;\nproperty id.account_balance string;\nproperty id.comment string;\n\ndatasource supplier (\n    s_suppkey:id,\n    s_name:name,\n    s_address:address,\n    s_nationkey:nation.id,\n    s_phone:phone,\n    s_acctbal:account_balance,\n    s_comment:comment\n)\ngrain (id)\naddress supplier;\n"
    },
    {
      "alias": "tutorial_one_basic",
      "contents": "# trilogy models run on imports to reuse logic\nimport lineitem as line_item;\n\n# you can define new concepts in line \nauto discounted_price <- line_item.extended_price * (1-line_item.discount); #the discounted price is off the extended privce\nauto charge_price <- discounted_price * (1+line_item.tax); #charged price includes taxes\n\n# and then use them in queries\n# use the run button or ctrl-enter to run this\nWHERE line_item.ship_date <= '1998-12-01'::date \nSELECT\n    line_item.return_flag,\n    line_item.line_status,\n    sum(line_item.quantity)-> sum_qty,\n    sum(line_item.extended_price)-> base_price,\n    sum(discounted_price) as sum_disc_price,\n    sum(charge_price) as sum_charge,\n    avg(line_item.quantity)-> avg_qty,\n    avg(line_item.extended_price)-> avg_price,\n    avg(line_item.discount)-> avg_discount,\n    avg(discounted_price)->avg_discounted_price,\n    # you can reuse a concept immediately in the same query\n    avg_price-avg_discounted_price as avg_disc,\n    count(line_item.id) as count_order\nORDER BY   \n    line_item.return_flag desc,\n    line_item.line_status desc\n;\n\n# check out the generated sql\n# the second tutorial editor will have a more complicated joins scheme\n# use the browser on the left to navigate to the other editor under the demo connection"
    },
    {
      "alias": "tutorial_three_function_agg",
      "contents": "# trilogy models run on imports to reuse logic\nimport lineitem as line_item;\n\n# you can define new concepts in line \nauto discounted_price <- line_item.extended_price * (1-line_item.discount); #the discounted price is off the extended privce\nauto charge_price <- discounted_price * (1+line_item.tax); #charged price includes taxes\n\n# use functions to define repeatable templatized logic\ndef part_percent_of_nation(x) -> sum(x) by line_item.part.name, line_item.supplier.nation.id / sum(x) by line_item.supplier.nation.id *100;\n\n\nWHERE line_item.ship_date <= '1998-12-01'::date \nSELECT\n    line_item.part.name,\n    line_item.supplier.nation.name,\n    sum(charge_price)-> total_charge_price,\n    sum(discounted_price) -> total_discounted_price,\n    # call functions with @\n    @part_percent_of_nation(charge_price) as charge_price_percent_of_nation,\n    @part_percent_of_nation(discounted_price) as discount_price_percent_of_nation\nORDER BY   \n    total_charge_price desc limit 100\n;\n\n"
    },
    {
      "alias": "tutorial_two_aggregate",
      "contents": "import part as part;\n\nWHERE part.supplier.nation.region.name = 'EUROPE'\nSELECT\n\tpart.supplier.account_balance,\n\tpart.supplier.name,\n\tpart.supplier.nation.name,\n\tpart.id,\n\tpart.manufacturer,\n\tpart.supplier.address,\n\tpart.supplier.phone,\n\tpart.supplier.comment,\n\t--part.supply_cost,\n\tmin(part.supply_cost) by part.id as min_part_cost,\nHAVING\n\tpart.supply_cost = min_part_cost\nORDER BY\n\tpart.id asc;"
    },
    {
      "alias": "trilogy-new-demo-model-connection_1745593636473",
      "contents": "CREATE OR REPLACE TABLE region AS \r\nSELECT * \r\nFROM read_parquet('https://shell.duckdb.org/data/tpch/0_01/parquet/region.parquet');\r\n\r\nCREATE  OR REPLACE  TABLE supplier AS\r\nSELECT * \r\nFROM read_parquet('https://shell.duckdb.org/data/tpch/0_01/parquet/supplier.parquet');\r\n\r\nCREATE  OR REPLACE  TABLE nation AS\r\nSELECT * \r\nFROM read_parquet('https://shell.duckdb.org/data/tpch/0_01/parquet/nation.parquet');\r\n\r\nCREATE  OR REPLACE  TABLE customer AS\r\nSELECT * \r\nFROM read_parquet('https://shell.duckdb.org/data/tpch/0_01/parquet/customer.parquet');\r\n\r\nCREATE  OR REPLACE  TABLE part AS\r\nSELECT * \r\nFROM read_parquet('https://shell.duckdb.org/data/tpch/0_01/parquet/part.parquet');\r\n\r\nCREATE OR REPLACE  TABLE supplier AS\r\nSELECT * \r\nFROM read_parquet('https://shell.duckdb.org/data/tpch/0_01/parquet/supplier.parquet');\r\n\r\nCREATE  OR REPLACE  TABLE partsupp AS\r\nSELECT * \r\nFROM read_parquet('https://shell.duckdb.org/data/tpch/0_01/parquet/partsupp.parquet');\r\n\r\n\r\nCREATE   OR REPLACE  TABLE orders AS\r\nSELECT * \r\nFROM read_parquet('https://shell.duckdb.org/data/tpch/0_01/parquet/orders.parquet');\r\n\r\nCREATE OR REPLACE   TABLE lineitem AS\r\nSELECT * \r\nFROM read_parquet('https://shell.duckdb.org/data/tpch/0_01/parquet/lineitem.parquet');"
    }
  ]
}

In [12]:
import datetime

request = QueryInSchema(
    query = '''
select 1-> test;''',
    dialect='bigquery',
    imports= [],
    full_model = ModelInSchema(**full_model),
)
start = datetime.datetime.now()
print(request.model_dump_json())
x = post('http://localhost:5678/generate_query', data=request.model_dump_json()).json()
print(datetime.datetime.now() - start)
print(x)
print(x)


{"imports":[],"query":"\nselect 1-> test;","dialect":"bigquery","full_model":{"name":"","sources":[{"alias":"customer","contents":"import std.money;\nimport nation as nation;\n\nkey id int;\nproperty id.name string;\nproperty id.address string;\nproperty id.phone string;\nproperty id.account_balance float::usd;\nproperty id.market_segment string;\nproperty id.comment string;\n\ndatasource customers (\n    c_custkey:id,\n    c_name:name,\n    c_address:address,\n    c_nationkey:nation.id,\n    c_phone:phone,\n    c_acctbal:account_balance,\n    c_mktsegment:market_segment,\n    c_comment:comment\n)\ngrain (id)\naddress customer;"},{"alias":"lineitem","contents":"import std.money;\nimport std.display;\nimport order as order;\nimport supplier as supplier;\nimport part as part;\n\n\n\nkey id int; #unique key identifying a lineitem within an order\nproperty id.quantity float; #quantity of an part within an order\nproperty id.extended_price float::usd;\nproperty id.discount float::percent; #

In [13]:
class MultiQueryComponent(BaseModel):
    query: str
    extra_filters: Optional[list[str]] = None
    parameters: Optional[dict[str, str | int | float]] = None
    
class MultiQueryInSchema(BaseModel):
    imports: List[Import]
    full_model: ModelInSchema
    dialect: Dialects
    queries: List[MultiQueryComponent]

In [14]:
queries = MultiQueryInSchema(
    imports=[],
    full_model=ModelInSchema(**full_model),
    dialect='bigquery',
    queries=[
        MultiQueryComponent(query='select 1-> test;'),
        MultiQueryComponent(query='select 2-> test;'),
        MultiQueryComponent(query='select 3-> test;'),
    ]

)

In [21]:
start = datetime.datetime.now()
print(request.model_dump_json())
x = post('http://localhost:5678/generate_queries', data=queries.model_dump_json()).json()
print(datetime.datetime.now() - start)
print(x)
for query, val in x.get('queries', []).items():
    print(query)
    print(val)

{"imports":[],"query":"\nselect 1-> test;","dialect":"bigquery","full_model":{"name":"","sources":[{"alias":"customer","contents":"import std.money;\nimport nation as nation;\n\nkey id int;\nproperty id.name string;\nproperty id.address string;\nproperty id.phone string;\nproperty id.account_balance float::usd;\nproperty id.market_segment string;\nproperty id.comment string;\n\ndatasource customers (\n    c_custkey:id,\n    c_name:name,\n    c_address:address,\n    c_nationkey:nation.id,\n    c_phone:phone,\n    c_acctbal:account_balance,\n    c_mktsegment:market_segment,\n    c_comment:comment\n)\ngrain (id)\naddress customer;"},{"alias":"lineitem","contents":"import std.money;\nimport std.display;\nimport order as order;\nimport supplier as supplier;\nimport part as part;\n\n\n\nkey id int; #unique key identifying a lineitem within an order\nproperty id.quantity float; #quantity of an part within an order\nproperty id.extended_price float::usd;\nproperty id.discount float::percent; #